In [216]:
import pycld2
import bs4
import re
import json
import datetime

In [60]:
with open("religioussystemo00call_djvu.xml") as f:
    soup = bs4.BeautifulSoup(f, "xml")

In [150]:
# Page regions (corresponds to chapters in book)

pagerngs = {
    "Unkulunkulu": (9, 107),
    "Utikxo": (122,124),
    "Lord of heavens": (126,131),
    "Amatongo": (137, 234),
    "Dreams": (236, 260),
    "Inkosazana": (261, 264),
    "Izinyanga Zokubula": (267,382),
    "Heaven-doctors": (383, 421),
    "Abatakati": (425, 456)
}

In [161]:
def append_text(t, a, unite_char=" "):
    """If previous paragraph ends with '-', complete the last word with the new string. Otherwise,
    append it along as new line/paragraph."""
    
    if t == "":
        return a
    elif t.endswith("-"):
        return t[:-1] + a
    else:
        return t + unite_char + a

In [87]:
def get_content(region):
    "Get the text content of the xml tag."
    
    r = ""
    for par in region.find_all("PARAGRAPH"):
        if not r.endswith("-") and r != "":
            r += "\n\n"
        for ln in par.find_all("LINE"):
            lntxt = " ".join(i.get_text() for i in ln.find_all("WORD"))
            r = append_text(r, lntxt)
    return r.strip()

In [183]:
def get_split(page):
    "Find the mid-page split between columns."
    
    xmin = xmax = []
    for r in page.find_all("REGION"):
        x1,y1,x2,y2 = get_rect(r)
        xmin.append(x1)
        xmax.append(x2)
    return min(xmin) + int((max(xmax)-min(xmin))/2)
    
def get_rect(region):
    
    x1, y1, x2, y2, y3 = [int(i) for i in region.find("WORD").attrs["coords"].split(",")]
    
    for w in region.find_all("WORD"):
        wx1, wy1, wx2, wy2, wy3 = [int(i) for i in w.attrs["coords"].split(",")]
        x1 = min(x1, wx1)
        y1 = max(y1, wy1)
        x2 = max(x2, wx2)
        y2 = min(y2, wy2)
    
    return (x1, y1, x2, y2)

In [212]:
def get_lang(region):
    "Use cld2 to infer language for the text in the xml tag."
    
    reliable, txtbytes, scores = pycld2.detect(get_content(region))
    return reliable, scores[0][1]

In [209]:
def region_pos(region):
    """Find out whether the xml tag is situated is on the left column (1), right column (3)
    or goes across the mid-page split (2)."""
    
    split = get_split(region.find_parent("OBJECT"))
    x1,y1,x2,y2 = get_rect(region)
    
    if x2 <= split:
        return 1 # Left of center split
    elif x1 < split and x2 > split:
        return 2 # Across center
    elif x1 >= split:
        return 3 # Right of center

In [225]:
def is_top(region):
    "Finds out whether it's the page header."
    x1, y1, x2, y2 = get_rect(region)
    px1, py1, px2, py2 = get_rect(region.find_parent("OBJECT"))
    
    return (y1 <= py2 + 70)

In [229]:
author = "Henry Callaway"
pubdate = 1870


# Roll through chapters
for chpname, pages in pagerngs.items():
    
    text = "" # New chapter → reinitialise text
    
    # Roll through pages
    for pagenum  in range(pages[0], pages[1]+1):
        # Load the specific page
        page = soup.find("OBJECT", usemap="religioussystemo00call_%04d.djvu" % pagenum)
        
        pagetxt = []  # We'll be putting text content in there
        
        # Roll through region tags
        for r in page.find_all("REGION"):
            pos = region_pos(r)
            
            if is_top(r): continue # Ignore if we're in the header
            
            langrel, lang = get_lang(r)
            rtxt = get_content(r)
            
            # Ignore if it looks like a header, title or page number, or if it's in English
            if rtxt.isupper() or rtxt.isalpha() or (lang=="en" and langrel):
                continue
                
            # Now to adding text
            if pos == 1 and lang == "zu" and langrel: # Safe case
                pagetxt.append(get_content(r))
                
            elif pos == 1 and lang == 'zu' and not langrel: # Uncertain language call
                print("**Uncertain language call on page %d (IsiZulu)**" % pagenum)
                print("\t>> p=%d, lang='%s', rel=%s" % (pos, lang, langrel))
                print("\t%s" % get_content(r))
                pagetxt.append(get_content(r)) # We still save it
                
            elif pos == 1 and lang != 'zu': # Text is on the left column, but language seems wrong
                print("**IsiZulu expected, but different language call on page %d (%s, rel=%s)**" % (pagenum, lang, langrel))
                print("\t%s" % get_content(r))
                pagetxt.append(get_content(r)) # We still save it
                
            elif pos > 1 and lang == 'zu': # Not in left column → we don't save it
                print("**Language called is IsiZulu (rel: %s) but not left bound on page %d (ignored)**" % (langrel,pagenum))
                print("\t%s" % get_content(r))
            
        text = append_text(text, "\n\n".join(pagetxt), unite_char="\n\n") # Append this page
    
    # Save it
    fname = "southafrica_folktale_religioussystem/southafrica_folktale_religioussystem_" + chpname.replace(" ", "_").lower()
    
    # Save the content
    with open(fname + ".txt", "w") as f:
        f.write(text)
    
    # Save the metadata
    metadata = {
        "title": "Religious system of the Amazulu: " + chpname,
        "author": author,
        "date_published": str(pubdate),
        "date_retrieved": datetime.datetime.now().strftime("%Y%M%d"),
        "url": "https://archive.org/details/religioussystemo00call"
    }
    
    with open(fname + ".json", "w") as f:
        json.dump(metadata, f)

**Uncertain language call on page 11 (IsiZulu)**
	>> p=1, lang='zu', rel=False
	Ku tiwa wa turn a unwaba ; wa ti, " Hamba, lunwaba, u ye ii yokuti, Abautu ina ba nga fi." Lwa hamba unwaba, hva hamba kancinane, lwa libala end/deleiii ; lwa hamba lwa dAla umuti, o igama lawo ku ubukwebezane.^

 Wa za Unkulunkulu wa tuma intulo ngasemva kwonwaba, se lu hambile ngesikati esipambili unwaba. Ya hamba intulo, ya gijima, ya tshetsha kakulu, ngokuba Unkulunkulu e tize, " Ntulo, u fike u ti, Abantu a ba fe," Ya hamba ke intulo, ya ti, " Ngi ti, Ku tiwa, Abantu ma ba fe." Ya buya intulo, ya fika kunkulunkulu ; lwa ba unwaba lu nga ka fiki, lona lwa tuny wa kukg-ala ; lona lwa tunywa ku tiwa, ma lu yokuti, " Abantu ma ba nca fi."
**IsiZulu expected, but different language call on page 12 (un, rel=False)**
	Lwa za Iwa fika, Iwa memeza, Iwa ti, " Ku tiwa, Abantu ma ba nga fi !" Ba ti abantu ba ti, " O ! si bambe izwi lentulo ; yona i si tshelile, ya ti, * Ku tiwa, Abantu ma ba fe.* A si sa li zwa el

**IsiZulu expected, but different language call on page 29 (un, rel=False)**
	Umpengula Mbanda.
**IsiZulu expected, but different language call on page 35 (un, rel=False)**
	koMwe ukuba si nga ti ni ngoiikulunkukx ; loku si nga s' azi nokwa/ilukana kwetu naye, nezwi a si shiya nalo. I ngaloko si zifunela aniad/tlozi, ukiize si libale si nga /ilali si kcamauga ngonkulunkulu, nkuti, " Unknlunkula wa si shiya;" nokuti, " U s' enzele ni na?"
**IsiZulu expected, but different language call on page 35 (un, rel=False)**
	Sa zenzela ke amad/ilozi etu, nabanye awabo, nabanye awabo. Se si fulatelene abanye nabanye ; a ku se ko o ti, " DAlozi lakwabani." Bonke se be ti, " DAlozi lakwiti, ekutinitini, n ngi bheke.' Ku njalo ke ukuma kwetu.
**IsiZulu expected, but different language call on page 35 (un, rel=False)**
	Na kulawo 'madAlozi a si nasiniiuya ; ngoba na labo 'bantu e si ba bougayo, si bonga abantu aba te nabo b' emuka kulo 'm/ilaba, ba be nga vumi ukumuka, ba b' ala kakulu, be si kataza n

**IsiZulu expected, but different language call on page 53 (xh, rel=True)**
	Si nga sa vela elu/ilaugeni ; a s' azi lapa sa bunjwa kona. Tina bantu 'bamnyama sa vela kunye nani 'belungu. Kodwa tina 'bantu 'bamnyama ukuvela kwetu sa vela sa nikwa izinkomo namagejo okulima ngemikono nezikali zokulwa. Kwa tiwa ke, " Okuningi ; se ni ya 'kuzenzela." S' emuka ke, s' eza neno. Nina 'belungu na sala nezinto zonke ezin/Je nemiteto futi e si nga banga nayo tina.
**IsiZulu expected, but different language call on page 53 (xh, rel=True)**
	Sa si va uma si i zekelwa bobaba, be ti nabo ba i va, ba ti, kwa k^'ala kwa vela umuntu o indoda ; kwa vela emuva umfazi. Kwa ti ngemva kwa vela inkomo ; ya vela i kamba nenkunzi ; kwa ti emva injakazana, kwa ti emva kwa vela inja e induna ; kwa ti ngemva zonke ke izilwanyane ezincinane lezi, nezind/dovu, zi vela ngambili njalo.

 Kwa ti ngemva kwa vela 'libele ; li ti 'libele uba li vele li ti nya, "wa ti lo 'muntu kumfazi, " Ku 'nto o ku bona nje ke, mfazi nd

**IsiZulu expected, but different language call on page 74 (un, rel=False)**
	nsvv'e kona aba d/iluhiko kuso isivivane, amancane amatshe namakulu e ponswa kona, ku tiwa, " Sivivane saokoko, ngi ti ketsheketshe nkuhamba kalula."

 Umpengula Mbanda.
**Uncertain language call on page 75 (IsiZulu)**
	>> p=1, lang='zu', rel=False
	lunkulu nal" Ukupendula kukalangeni, wa ti, " E tslio umuntii wokiik^'ala kiibo boiike abantu, owa vezwa Utiku:;o kuk(^ala. Kepa abautu ba m bona. Utika;o wa sita kuukvJunkiilu, ka bonwaiiga iimuntu ; abantu ba bona yena TJnkulunkulu, ba ti umenzi wako konke, XJmvelinyangi, be tslio ngokuba lowo ow' enza Unkulunkulu be nga m bonanga. Ba ti ke u yena e Utikso. I loko e ngi k\v aziyo ngonkulunkulu."
**IsiZulu expected, but different language call on page 76 (un, rel=False)**
	ku ng' aziwa uma u ti ni na, e lal' end/ile, e nga lali ekaya ; kepa lima e bona umuzi a ye kuwo ; nakuba ukukuluraa kwabantu e uga kw azi, a k^vitize njalo kubantu, ba koAlwe uaia ii ti ni na.

**Language called is IsiZulu (rel: False) but not left bound on page 97 (ignored)**
	komlia amabele e uiilile, wa ti, "Ka-iiini.^* Nank' amabele," e tsho Unkulunkulu wamandulo,^^ Ukgili. TJkgili kambe Unkulunkulu ■wokuk^'ala owa puma emAlangeni, wa zala abantu.

 TJmuntu wokuk^fala u tiwa Unkulunkulu. Wa vela nomfazi ; nabanye abantu ba vela em/ilangeni emva kwake, abantu bonke bendulo. Yena owokuk^'ala e mkulu kambe, yena a zala abantu. Si tsbo tina 'bantu, si ti, " Ba zaIwa nguye yena a vela kukgala."

 Abantu bendulo a si b' azi ukuzalwa kwabo. Ba vela em/Jangeni nje ; naye Unkulunkulu wa vela

 ^* Ka-nini, Pluck, for Yika ni.

 ^^ Wamandulo. — The most ancient Unkulunkulu

 Ba kona abantu bendulo abaningi, kepa e si ti owaraandulo o ngapambili kwalabo bendulo.
**IsiZulu expected, but different language call on page 97 (xh, rel=True)**
	Ku tshiwo iimuutu ow azi kakulu ; ngaloko ke ngoku/Jakanipa kwake a ku sa tshiwo ukuti ik^'ili, se ku tiwa Uk^'ili. Owokuk(7ala ku tiwa Uk^^ili, ngo

...

**IsiZulu expected, but different language call on page 392 (un, rel=False)**
	e lu pete wa lu kumiila esikwameni sake ; nga bona nembala, nga tsho iikuti, " Hau ! olwenyoni esabekayo." Wa ngi bonisa netambo layo ; la fana netambo li fakwe umtanjana omuncinyane wegazi nomtsliwana o Alangana nompofana ; nga bona imitshwe eminingi etanjeni layo, nga ti, "Nembala." I loko ke e nga ku zwa ngaleyo 'ndaba. Kwa pela ngaloko ke, e nga zibonela kona ngawami ameh\o.

 Izinyanga zokwalusa si kuluma iigomfanekiso, ngokiiba umuntii owalusa izinkomo u nezikali neg^okwe lemvula. Sa tata lelo 'gama lomalusi wezinkomo, si biza omelana nonyazi, ngoknba uma e lu tiba u ya memeza njengomfana weziukomo ; yena uma e ngena esibayeni nezikali zake, a tule nje, zi nge puma izinkomo ; kepa ngokuAlo/ila ikwelo, izinkomo zi y' ezwa iikuba n ti nga z' aluka, iiknti a zi pume esibayeni. Na lowo 'malusi owalusa nnyazi w enza njeugalowo wezinkomo; w enza njalo ke ngoku/JoAla ikwelo ; a ti, '' Tsliui-i-i. Hamba, u ye 

**IsiZulu expected, but different language call on page 404 (un, rel=False)**
	puma s' azi ke ukuba li bu gcinile ubusika, se li landa i/ilobo ; nenibala li ye li hambela ku ze ku ti ukukula kwe/tlobo li iigene end/ilini yase/tlobo iziusukwana, li pume njalo.
**IsiZulu expected, but different language call on page 404 (un, rel=False)**
	Inyanga ukutwasa kwayo si ti i twasile inyanga ngokuba si i bona entshonalanga. Kwa ku tiwa inyanga i fa nya ; kanti a ku njalo ; i dAliwa izinsuku, i ye i ncipa, i ze i be ngangozipo uje enie/ilweni ; lapo ke se i tatwa ilanga ; li i fumana empumalanga, li hambe nayo, li ze li i shiye entslionalaiiga, i bonwe lapa ku kgala ukuti zibe uku/ilwa, ku tiwe i twasile inyanga ; i ye i kula ; i ze i fulatele enzansi, si ti inyanga se i bheka enzansi, i ye i Alangane ukutshona kwelanga, i ze i selwe ; i buye i ncipe futi, i ze i fe.
**IsiZulu expected, but different language call on page 404 (xh, rel=True)**
	Izinkanyezi a si tslio ukuba zi ya hamba njengelanga

**IsiZulu expected, but different language call on page 414 (rw, rel=True)**
	u /ilangauise nemiuye ke imiti ; amagama ayo a ngi w' azi. I leyo ke e ngi y aziyo imiti yezulu.
**IsiZulu expected, but different language call on page 414 (xh, rel=True)**
	Ku ti inyanga ey alusa izulu ngam/ila i d/ila imifino yonyaka
**IsiZulu expected, but different language call on page 415 (un, rel=False)**
	omiitsha, Icii tiwe i y' csliwama, ku zilwe ku nga setslienzwa ; abantu ba /ilalc emakaya, ba nga sebenzi. Futi ku ti uma li wisa isik(7oto, ba nga sebenzi, ba zile ngokuti, " O, nma si sebenza si banga izulu." Konke loko a kw enziwa. Noma li vungiiza umoya ngesikati sokuba se ku linywa, ku njalo futi a ku linywa, ku ya zihva njalo, ngokuti, " Uma si lima si ya zibangela. Ku/tle ukuba si zile, kona umoya ngomso u nga yi 'kufika ngamauclAla."
**IsiZulu expected, but different language call on page 415 (un, rel=False)**
	I loko ke e ngi kw aziyo ngezulu. Kepa imiti yona e patwa izinyanga miningi, emin

**IsiZulu expected, but different language call on page 433 (un, rel=False)**
	wa zibika ukuti, "Na kumi ku njalo." Kepa leyo 'nyama a ba i k^yedanga, ba ba se be bulawa na amakanda; kwa ba se ku ukungena kwokugula iijalo.

 Lokupela Aman/dwenga a ya s' azi leso 'sifo uku s' elapa. Unofi/tlela wa kqala, ukuk^/umba, isisu a sa be si sa pela ; wa ba loku 'esuti njalo. Nomndava wa ba njalo ; kwa za kwa ba hhudisa loko 'kufa.

 U te umlungu wabo, um' a bone ukuba ku njalo, abantu be za 'kufa; lokupela baningi, kwa ba /daba bonke,ukupela umlungu lowo e ku nga m Alabanga ; (kauti naye wa ka wa gula pambili ngokuya kwake kwokukr/ala, w' elatshwa; kanti u se pinda ukuya;) wa biza izinyanga zakona, za b' elapa. Kepa ekufikeni kwabo kuman/dwenga,bafikaa}ianyebenga se ko, se be file. Kepa labo abab' elapayo ba ba tshela ukuti, " Ni nga wa puzi amanzi, futi ni nga wa d/di amasi ; uma ni d/da amasi, ni ya 'kufa ezind/deleni ; a ni yi 'kutika."

 Nembala ke, lokupela ba ti be s' elatsbwa, ibunu la t

**IsiZulu expected, but different language call on page 448 (xh, rel=True)**
	ndeki ; ku ya gwiy wa, kw apnlwa izinti ebusweni bayo. Ukwapula uti ebusweni benkosi kubantu abamnyarna ukufunga okukuhi, ukuti, " U ya 'u ngi buza, uma u ng' ezwanga indaba yami."
**IsiZulu expected, but different language call on page 451 (un, rel=False)**
	shiye." Kwa ku ng* enziwa, ngokuba ku tiw;v i ya tshetsha iaipi uku m /ilaba o gubuzelisileyo, uktiti uiDakoti. Ku ngaloko ke ku be kona iudoda e m tshelayo unilisa lowo, noma xiyise, a ti, " Wena, u iiga yi ; Mala," e tslio ngasesc.

 Kepa loko kwamazulu kwa pela ; iigokuba amabuto onke a ku taiidwaiiga ukuba a gaiiwe ; kwa tiwa bouke a ba nga ganwa, iikuze ba ng' esabi. Ku be ku njalo pakati kwetu ; kwa kw esatshwa uma uniuntu e puma impi e shiya 6 gobisile. Kwa yekwa ngokuti, " O, ai ! loku abautu ba ya 'kuti, * XJmuutu o gobisileyo u y' esaba ; Ti tanda umfazi kuuobukiyawe.' " Se ku pelile manje ; a ku se ko.
**IsiZulu expected, but different languag